In [ ]:
from pyModbusTCP.client import ModbusClient

# 建立 Modbus 客戶端
client = ModbusClient(host="127.0.0.1", port=502)


if not client.is_open:
    if not client.open():
        print("無法連接到 Modbus Server")
    else:
        print("已連接到 Modbus Server")


start = 0  # 起始位址
count = 10  # 要讀取的數量

result = client.read_holding_registers(start, count) # 讀取Register
# result = client.read_coils(start, count) # 讀取Coil

if result:
    print("讀取的值：", result)
else:
    print("Modbus 讀取失敗")

client.close()

In [ ]:
from pyModbusTCP.client import ModbusClient

# 建立 Modbus 客戶端
client = ModbusClient(host="127.0.0.1", port=502)

if not client.is_open:
    if not client.open():
        print("無法連接到 Modbus Server")
    else:
        print("已連接到 Modbus Server")

address = 5  # 要寫入的位址
value_to_write = 10  # 要寫入的值


result = client.write_single_register(address, value_to_write) # 寫入Register
# result = client.write_single_coil(address, value_to_write) # 寫入Coil

if result:
    print("寫入成功")
else:
    print("寫入失敗")

client.close()

In [ ]:
import scapy.all as scapy
import scapy.contrib.modbus as mb

for pkt in scapy.PcapReader("write.pcapng"):
    if mb.ModbusPDU06WriteSingleRegisterResponse in pkt:
        pkt.show()

In [ ]:
from scapy.all import *

# Modbus ADU
class ModbusTCP(Packet):
    name = "Modbus/TCP"
    fields_desc = [
        ShortField("Transaction_Identifier", 3),
        ShortField("Protocol_Identifier", 0),
        ShortField("Length", 6),
        XByteField("Unit_Identifier", 1),
    ]

# Modbus PDU
class Modbus(Packet):
    name = "Modbus"
    fields_desc = [
        XByteField("Function_Code", 6),  
        ShortField("Reference_Number", 53),
        # ShortField("Word_Count", 63),
        ShortField("Data", 1234),
    ]


s = socket.socket()
s.connect(("192.168.50.197", 502))
ss = StreamSocket(s, Raw)

try:
    response = ss.sr1(Raw(ModbusTCP()/Modbus()))
    if response:
        response_hexdump = hexdump(response, dump=True)
        print(response_hexdump)

except KeyboardInterrupt:
    pass

In [ ]:
from scapy.all import *

# Modbus ADU
class ModbusTCP(Packet):
    name = "Modbus/TCP"
    fields_desc = [
        ShortField("Transaction_Identifier", 3),
        ShortField("Protocol_Identifier", 0),
        ShortField("Length", 6),
        XByteField("Unit_Identifier", 1),
    ]

# Modbus PDU
class Modbus(Packet):
    name = "Modbus"
    fields_desc = [
        XByteField("Function_Code", 3),  
        ShortField("Reference_Number", 0),
        ShortField("Word_Count", 64),
        # ShortField("Data", 1234),
    ]


s = socket.socket()
s.connect(("192.168.50.197", 502))
ss = StreamSocket(s, Raw)

try:
    response = ss.sr1(Raw(ModbusTCP()/Modbus()))
    if response:
        response_hexdump = hexdump(response, dump=True)
        print(response_hexdump)

except KeyboardInterrupt:
    pass